<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/03_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Embeddings de Palabras usando Sherlock Holmes


# 1) Carga de Datos

Carga un texto desde Project Gutenberg y elimina el header/footer:

In [1]:
import urllib.request
from pathlib import Path

def load_gutenberg_text(url):
    print("Cargando texto desde:", url, "..")
    response = urllib.request.urlopen(url)
    raw = response.read().decode('utf-8')

    # Encontrar el inicio y fin del contenido real (eliminar header/footer de Gutenberg)
    start = raw.find("*** START OF THE PROJECT GUTENBERG")
    start = raw.find("\n", start) + 1
    end = raw.find("*** END OF THE PROJECT GUTENBERG")

    return raw[start:end]

# URLs de los libros en Project Gutenberg
urls = [
    "https://www.gutenberg.org/files/1661/1661-0.txt",    # The Adventures of Sherlock Holmes
    #"https://www.gutenberg.org/files/108/108-0.txt",      # The Return of Sherlock Holmes
    #"https://www.gutenberg.org/files/69700/69700-0.txt",  # The case-book of Sherlock Holmes
]

# Carga y guarda el texto
text = " "
for url in urls:
  book_text = load_gutenberg_text(url)
  text += book_text


print("Texto cargado:", len(text), "caracteres")

Cargando texto desde: https://www.gutenberg.org/files/1661/1661-0.txt ..
Texto cargado: 574154 caracteres


# 2) Preprocesamiento

Tokeniza y limpia el texto:
* Convierte a minúsculas
* Elimina tokens que no son palabras

In [2]:
import spacy
from spacy.lang.en import English

# Inicializar spaCy (solo tokenizador para velocidad)
nlp = English(disable=['tagger','parser','ner'])

def tokenize(text):
    doc = nlp(text)
    return [token.text.lower() for token in doc
            if token.text.strip() and not token.is_punct]

# Dividir en oraciones y tokenizar
corpus_sentences = []
for line in text.split('\n'):
    if line.strip():  # ignorar líneas vacías
        tokens = tokenize(line)
        if tokens:  # ignorar líneas sin tokens válidos
            corpus_sentences.append(tokens)

print("Total de oraciones procesadas:", len(corpus_sentences))

Total de oraciones procesadas: 9313


# 3) Entrenamiento de Modelos

In [3]:
from gensim.models import Word2Vec, FastText

Entrenar Word2Vec

In [4]:
w2v_model = Word2Vec(sentences=corpus_sentences,
                    vector_size=100,
                    window=5,
                    min_count=2,
                    workers=4)

Entrenar FastText

In [5]:
ft_model = FastText(sentences=corpus_sentences,
                   vector_size=100,
                   window=5,
                   min_count=2,
                   workers=4)

Guardar modelos

In [6]:
w2v_model.save("sherlock_w2v.model")
ft_model.save("sherlock_ft.model")

# 4) Análisis de Similitudes

Cargar vectores

In [7]:
w2v_vectors = w2v_model.wv
ft_vectors = ft_model.wv

print("\nEstadísticas de los modelos:")
print("Dimensión de los vectores:", w2v_vectors.vector_size)
print("Número de palabras (Word2Vec):", len(w2v_vectors.index_to_key))
print("Número de palabras (FastText):", len(ft_vectors.index_to_key))


Estadísticas de los modelos:
Dimensión de los vectores: 100
Número de palabras (Word2Vec): 4390
Número de palabras (FastText): 4390


In [8]:
print("\nPalabras más similares a 'holmes' (Word2Vec):")
print(w2v_vectors.most_similar('holmes'))


Palabras más similares a 'holmes' (Word2Vec):
[('wilson', 0.9974080920219421), ('lestrade', 0.9969411492347717), ('dear', 0.996678352355957), ('he', 0.996609091758728), ('mr.', 0.9964424967765808), ('she', 0.9964025616645813), ('pray', 0.996375322341919), ('let', 0.9963335394859314), ('help', 0.9962263703346252), ('far', 0.9962010979652405)]


In [9]:
print("\nPalabras más similares a 'crime' (Word2Vec):")
print(w2v_vectors.most_similar('crime'))



Palabras más similares a 'crime' (Word2Vec):
[('most', 0.9991777539253235), ('years', 0.9991291165351868), ('or', 0.9991259574890137), ('time', 0.9991074800491333), ('some', 0.9990922808647156), ('five', 0.9990889430046082), ('first', 0.9990875720977783), ('long', 0.9990733861923218), ('like', 0.9990726709365845), ('every', 0.9990718960762024)]


In [10]:
print("\nSimilitud entre 'holmes' y 'detective':")
print("Word2Vec:", w2v_vectors.similarity('holmes', 'detective'))
print("FastText:", ft_vectors.similarity('holmes', 'detective'))


Similitud entre 'holmes' y 'detective':
Word2Vec: 0.9788761
FastText: 0.9993339


# 5) Experimentos con palabras fuera de vocabulario

In [11]:
print("\nPrueba con palabra fuera de vocabulario:")
try:
    print("Word2Vec - Similares a 'investigador':")
    print(w2v_vectors.most_similar('investigador'))
except KeyError:
    print("Word2Vec no puede manejar palabras fuera de vocabulario")

print("\nFastText - Similares a 'investigador':")
print(ft_vectors.most_similar('investigador'))  # FastText puede generar vectores para palabras nuevas



Prueba con palabra fuera de vocabulario:
Word2Vec - Similares a 'investigador':
Word2Vec no puede manejar palabras fuera de vocabulario

FastText - Similares a 'investigador':
[('investigation', 0.9999861717224121), ('investigations', 0.9999841451644897), ('investments', 0.9999807476997375), ('invent', 0.9999798536300659), ('information', 0.9999767541885376), ('instrument', 0.9999763369560242), ('portion', 0.9999760389328003), ('plantation', 0.9999752640724182), ('inspection', 0.9999748468399048), ('instant', 0.9999748468399048)]


# 6) Analogías

In [12]:
print("\nAnalogías (Word2Vec):")
result = w2v_vectors.most_similar(positive=['holmes', 'crime'],
                                negative=['police'])
print("holmes:crime como police:?")
print(result)


Analogías (Word2Vec):
holmes:crime como police:?
[('he', 0.9967093467712402), ('rucastle', 0.9966807961463928), ('wilson', 0.9961453080177307), ('lestrade', 0.9957737326622009), ('also', 0.9957162141799927), ('st.', 0.9956408143043518), ('merryweather', 0.9955851435661316), ('john', 0.9955640435218811), ('always', 0.995510458946228), ('neville', 0.9954965114593506)]


# 7) Mejora del Preprocesamiento



Función mejorada de limpieza de texto:
* Elimina números
* Elimina puntuación
* Elimina texto entre paréntesis (acotaciones)
* Elimina múltiples espacios
* Normaliza apóstrofes y comillas

In [13]:
import re
import string

def clean_text(text):
    try:
        # Normalizar apóstrofes y comillas
        text = re.sub(r"[''']", "'", text)
        text = re.sub(r'["""]', '"', text)

        # Eliminar texto entre paréntesis
        text = re.sub(r'\([^)]*\)', '', text)

        # Eliminar números
        text = re.sub(r'\d+', '', text)

        # Eliminar puntuación preservando apóstrofes
        text = re.sub(r'[^\w\s\']', ' ', text)

        # Normalizar espacios
        text = re.sub(r'\s+', ' ', text)

        return text.strip()
    except Exception as e:
        print(f"Error en la limpieza del texto: {e}")
        return text

Procesa un token individual:
* Convierte a minúsculas
* Maneja contracciones comunes
* Elimina tokens muy cortos

In [14]:
def preprocess_token(token):
    # Convertir a minúsculas
    token = token.lower()

    # Diccionario de contracciones
    contractions = {
        "i'm": "i am",
        "he's": "he is",
        "she's": "she is",
        "it's": "it is",
        "that's": "that is",
        "what's": "what is",
        "when's": "when is",
        "where's": "where is",
        "who's": "who is",
        "why's": "why is",
        "i'll": "i will",
        "you'll": "you will",
        "he'll": "he will",
        "she'll": "she will",
        "it'll": "it will",
        "we'll": "we will",
        "they'll": "they will",
        "i'd": "i would",
        "you'd": "you would",
        "he'd": "he would",
        "she'd": "she would",
        "it'd": "it would",
        "we'd": "we would",
        "they'd": "they would",
        "won't": "will not",
        "can't": "cannot",
        "don't": "do not",
        "doesn't": "does not",
        "didn't": "did not",
        "haven't": "have not",
        "hasn't": "has not",
        "hadn't": "had not",
        "wouldn't": "would not",
        "couldn't": "could not",
        "shouldn't": "should not",
        "mightn't": "might not",
        "mustn't": "must not"
    }

    # Expandir contracciones
    if token in contractions:
        return contractions[token].split()

    # Eliminar tokens muy cortos (menos de 2 caracteres)
    if len(token) < 2:
        return []

    return [token]

Versión mejorada de tokenización:
* Usa la limpieza mejorada
* Maneja contracciones
* Opcionalmente elimina stopwords
* Elimina tokens no válidos

In [15]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def tokenize_improved(text, remove_stopwords=True):
    # Obtener stopwords
    stop_words = set(stopwords.words('english')) if remove_stopwords else set()

    # Limpiar el texto
    text = clean_text(text)

    # Tokenizar
    doc = nlp(text)

    # Procesar cada token
    tokens = []
    for token in doc:
        processed_tokens = preprocess_token(token.text)
        for t in processed_tokens:
            if remove_stopwords and t in stop_words:
                continue
            if t.strip() and not t.isnumeric():
                tokens.append(t)

    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Procesar el texto con la nueva función

In [16]:
improved_sentences = []
for line in text.split('\n'):
    if line.strip():
        tokens = tokenize_improved(line)
        if tokens:
            improved_sentences.append(tokens)

print("Total de oraciones procesadas (mejorado):", len(improved_sentences))


Total de oraciones procesadas (mejorado): 9220


## 8) Entrenamiento de Modelos

In [17]:
w2v_model = Word2Vec(sentences=improved_sentences,
                    vector_size=100,
                    window=5,
                    min_count=2,
                    workers=4)

# 9) Análisis de Similitudes

In [18]:
print("\nPalabras más similares a 'holmes' (Word2Vec):")
print(w2v_vectors.most_similar('holmes'))


Palabras más similares a 'holmes' (Word2Vec):
[('wilson', 0.9974080920219421), ('lestrade', 0.9969411492347717), ('dear', 0.996678352355957), ('he', 0.996609091758728), ('mr.', 0.9964424967765808), ('she', 0.9964025616645813), ('pray', 0.996375322341919), ('let', 0.9963335394859314), ('help', 0.9962263703346252), ('far', 0.9962010979652405)]
